In [1]:
import numpy as np
import pandas as pd

import os, urllib, time
os.environ["MLFLOW_TRACKING_URI"] = "sqlite:///mlruns/db.sqlite"
import mlflow, git
mlflow_client = mlflow.tracking.MlflowClient()

In [12]:
def leaders(NJ, name):
    experiment_id = "0"
    run_name = f"eos.{NJ}J {name}"

    rank_metrics = ['|alpha_tmu|', '|eyeerr_tmu|', '-reward_total']
    print_metrics = ['model_version', 'alpha_tmu', 'alpha_tsigma', 'alpha_tdf', 'eyeerr_tmu', 'eyeerr_tsigma', 'eyeerr_tdf', 'reward_total']

    parent_runs = mlflow_client.search_runs(experiment_id, f"tags.mlflow.runName='{run_name}'")

    best_metric = dict()
    best_run = dict()

    for parent_run in parent_runs:
        child_runs = mlflow_client.search_runs(experiment_id, f"tags.mlflow.parentRunId='{parent_run.info.run_id}'")
        #                                           order_by=["metrics.reward_total DESC"])
        for child_run in child_runs:
            if 'alpha_tmu' not in child_run.data.metrics:
                continue

            child_run.data.metrics['|alpha_tmu|'] = np.abs(child_run.data.metrics['alpha_tmu'])
            child_run.data.metrics['|eyeerr_tmu|'] = np.abs(child_run.data.metrics['eyeerr_tmu'])
            child_run.data.metrics['-reward_total'] = - child_run.data.metrics['reward_total']

            for metric in rank_metrics:
                if metric not in best_metric or child_run.data.metrics[metric] < best_metric[metric]:
                    best_metric[metric] = child_run.data.metrics[metric]
                    best_run[metric] = child_run
    metric_values = []
    for whats_best, run in best_run.items():
        best_values = [run.data.metrics[metric] for metric in print_metrics]
        metric_values.append([whats_best] + best_values)
    return pd.DataFrame(metric_values, columns=['whats_best'] + print_metrics)                        

In [13]:
leaders(3, "no eye_phi obs")

,whats_best,model_version,alpha_tmu,alpha_tsigma,alpha_tdf,eyeerr_tmu,eyeerr_tsigma,eyeerr_tdf,reward_total
0,|alpha_tmu|,213.0,0.000762,7.652273e-04,0.418541,-0.001629,1.835374e-03,0.436647,117.0
1,|eyeerr_tmu|,213.0,0.000762,7.652273e-04,0.418541,-0.001629,1.835374e-03,0.436647,117.0
2,-reward_total,206.0,-0.003831,3.705622e-09,0.093428,0.006333,1.676402e-08,0.121492,126.0
